<a href="https://colab.research.google.com/github/KIM-CHAEYOUNG/PROJECT_COVID19_Delivery/blob/master/%5BSTUDY%7D%ED%99%95%EC%A7%84%EC%9E%90_%EC%A7%80%EC%97%AD%EB%8D%B0%EC%9D%B4%ED%84%B0_%EC%A0%84%EC%B2%98%EB%A6%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 코로나 관련 이동성 데이터 수집 및 분석
[참고사이트](https://dacon.io/competitions/official/235622/codeshare/1606?page=1&dtype=recent&ptype=pub) : [DACON] 국토교통 빅데이터 온라인 해커톤 경진대회

- 코로나 관련 이동성 데이터 수집 및 분석

# 라이브러리 및 데이터
### [COVID-19 데이터]

*   Patientinfo.csv
*   Case.csv

### [교통카드 데이터]

*   교통수단별 이용량 (DM_TFBY_USEV_SMY.csv)
*   평균통행시간 (DM_AVGTRFCTM_T.csv)

### [외부 데이터]


*   [subway.csv](https://data.seoul.go.kr/dataList/OA-12252/S/1/datasetView.do)
*   항목 추가
*   항목 추가






In [46]:
from datetime import datetime
from collections import Counter

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [17]:
from google.colab import drive

ROOT = "/content/drive"
print(ROOT)
drive.mount(ROOT)

/content/drive
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [24]:
from os.path import join  

MY_GOOGLE_DRIVE_PATH = 'My Drive/Colab Notebooks/COVID_19' # 프로젝트 경로
PROJECT_PATH = join(ROOT, MY_GOOGLE_DRIVE_PATH) # 프로젝트 경로
print(PROJECT_PATH)

/content/drive/My Drive/Colab Notebooks/COVID_19


In [28]:
%cd "{PROJECT_PATH}"

/content/drive/My Drive/Colab Notebooks/COVID_19


In [65]:
df_patient=pd.read_csv('PatientInfo.csv')
df_case=pd.read_csv('Case.csv')
df_region=pd.read_csv('Region.csv')

In [66]:
df_patient.head()

,Unnamed: 0,patient_id,sex,age,country,province,city,infection_case,infected_by,contact_number,symptom_onset_date,confirmed_date,released_date,deceased_date,state,confirmed_dt,patiend_id,valid
0,0,1.000000e+09,male,50s,Korea,서울,강서구,overseas inflow,NaN,75,2020-01-22,2020-01-23,2020-02-05,NaN,released,2020-01-23,NaN,1
1,1,1.000000e+09,male,30s,Korea,서울,중랑구,overseas inflow,NaN,31,NaN,2020-01-30,2020-03-02,NaN,released,2020-01-30,NaN,1
2,2,1.000000e+09,male,50s,Korea,서울,종로구,contact with patient,2002000001,17,NaN,2020-01-30,2020-02-19,NaN,released,2020-01-30,NaN,1
3,3,1.000000e+09,male,20s,Korea,서울,마포구,overseas inflow,NaN,9,2020-01-26,2020-01-30,2020-02-15,NaN,released,2020-01-30,NaN,1
4,4,1.000000e+09,female,20s,Korea,서울,성북구,contact with patient,1000000002,2,NaN,2020-01-31,2020-02-24,NaN,released,2020-01-31,NaN,1


In [67]:
df_region.head()

,code,province,city,latitude,longitude,elementary_school_count,kindergarten_count,university_count,academy_ratio,elderly_population_ratio,elderly_alone_ratio,nursing_home_count
0,10000,서울,서울,37.566953,126.977977,607,830,48,1.44,15.38,5.8,22739
1,10010,서울,강남구,37.518421,127.047222,33,38,0,4.18,13.17,4.3,3088
2,10020,서울,강동구,37.530492,127.123837,27,32,0,1.54,14.55,5.4,1023
3,10030,서울,강북구,37.639938,127.025508,14,21,0,0.67,19.49,8.5,628
4,10040,서울,강서구,37.551166,126.849506,36,56,1,1.17,14.39,5.7,1080


In [76]:
# 지역 데이터도 지역명 수정
def fix_region(row):
    fixdic = {
        ('경기도','구리'):('경기도','구리시'),
        ('경기도','수원'):('경기도','수원시'),
        ('경기도','양평시'):('경기도','양평'),
        ('대구','경산시'):('경상북도','경산시'),
        ('대구','달성군'):('대구','달성'),
        ('대구','상곡 동'):('대구','대구'),
        ('대구','영천시'):('경상북도','영천시'),
        ('대구','이천동'):('대구','남구'),
        ('대전','세종'):('세종','세종'),
        ('전라남도','경산시'):('경상북도','경산시'),
        ('경상남도','하만'):('경상남도','함안군'),
        ('서울','동구 구'):('서울','동작구'),
        ('인천','미추홀 구'):('인천','미추홀구'),
        ('전라북도','지난'):('전라북도','진안군'),
        ('전라남도','시난'):('전라남도','신안군')
    }
    if (row.province, row.city) in fixdic.keys():
        row.province, row.city = fixdic[(row.province, row.city)]
    if (row.province == row.city) or (row.city == '기타'):
        #print((row.province,row.city))
        row.city = np.nan
    return row
df_region=df_region.apply(fix_region, axis=1)

In [77]:
df_region['id']=df_region.apply(lambda x: (x['province'], x['city']), axis=1)
# id라는 컬럼에 'province'와 'city'의 컬럼내용을 넣음
# '서울' '강남구' -> (서울, 강남구)

In [78]:
df_region.head()

,code,province,city,latitude,longitude,elementary_school_count,kindergarten_count,university_count,academy_ratio,elderly_population_ratio,elderly_alone_ratio,nursing_home_count,id
0,10000,서울,NaN,37.566953,126.977977,607,830,48,1.44,15.38,5.8,22739,"(서울, nan)"
1,10010,서울,강남구,37.518421,127.047222,33,38,0,4.18,13.17,4.3,3088,"(서울, 강남구)"
2,10020,서울,강동구,37.530492,127.123837,27,32,0,1.54,14.55,5.4,1023,"(서울, 강동구)"
3,10030,서울,강북구,37.639938,127.025508,14,21,0,0.67,19.49,8.5,628,"(서울, 강북구)"
4,10040,서울,강서구,37.551166,126.849506,36,56,1,1.17,14.39,5.7,1080,"(서울, 강서구)"


In [71]:
# 위치 추출
coords = df_region[['id', 'longitude', 'latitude']]

In [72]:
coords.head()

,id,longitude,latitude
0,"(서울, nan)",126.977977,37.566953
1,"(서울, 강남구)",127.047222,37.518421
2,"(서울, 강동구)",127.123837,37.530492
3,"(서울, 강북구)",127.025508,37.639938
4,"(서울, 강서구)",126.849506,37.551166


In [73]:
coords.shape

(244, 3)

In [74]:
pos ={('해외', np.nan):(123,37)}

In [80]:
for i, row in coords.iterrows():
    pos[row['id']]= (row['longitude'], row['latitude'])

In [84]:
pos

{('강원도', '강릉시'): (128.87592800000002, 37.75197),
 ('강원도', '고성'): (128.467827, 38.380571),
 ('강원도', '동해시'): (129.114239, 37.524689),
 ('강원도', '삼척시'): (129.16536499999998, 37.449899),
 ('강원도', '속초'): (128.59186100000002, 38.207022),
 ('강원도', '양구'): (127.990092, 38.110002),
 ('강원도', '양양'): (128.619125, 38.075405),
 ('강원도', '영월'): (128.461835, 37.183694),
 ('강원도', '원주시'): (127.919668, 37.341963),
 ('강원도', '인제'): (128.17033500000002, 38.069682),
 ('강원도', '정선군'): (128.660873, 37.38062),
 ('강원도', '철원'): (127.3134, 38.146693),
 ('강원도', '춘천시'): (127.730088, 37.881281),
 ('강원도', '태백시'): (128.985736, 37.16406),
 ('강원도', '평창'): (128.39033500000002, 37.370743),
 ('강원도', '홍천'): (127.888821, 37.697037),
 ('강원도', '화천'): (127.70816299999998, 38.106152),
 ('강원도', '횡성'): (127.985042, 37.491702),
 ('강원도', nan): (127.729868, 37.885369),
 ('경기도', '가평'): (127.509555, 37.831297),
 ('경기도', '고양시'): (126.831961, 37.658363),
 ('경기도', '과천시'): (126.987616, 37.42915),
 ('경기도', '광명시'): (126.864648, 37.478541),
 ('경기도